Batch run contoso-chat

In [ ]:
from promptflow import PFClient
from promptflow.entities import Run

pf_client = PFClient()

In [ ]:
# load flow
flow = "../contoso-chat"
# load data
data = "../data/salestestdata.jsonl"

In [ ]:
# get current time stamp for run name
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y_%m_%d_%H%M%S")
run_name = timestamp+"chat_base_run"
print(run_name)

In [ ]:
# Get a pf client to manage runs
pf = PFClient()

# Initialize an Run object
base_run = Run(
    flow=flow,
    # run flow against local data or existing run, only one of data & run can be specified.
    data=data,
    #run="<existing-run-name>",
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
    },
    #variant="${summarize_text_content.variant_0}"
)

# Create the run
result = pf.runs.create_or_update(base_run)

print(result)

In [ ]:
result_df = pf_client.get_details(base_run.name, all_results=True)

In [ ]:
result_df.head(10)

Run contoso-chat evaluation

In [ ]:
import os
from azure.ai.generative.evaluate import evaluate
import json
import numpy as np
from azure.ai.resources.client import AIClient
from azure.identity import InteractiveBrowserCredential, DefaultAzureCredential
from azure.ai.resources.entities import AzureOpenAIModelConfiguration
from azure.ai.generative.evaluate.metrics import PromptMetric
import nest_asyncio
nest_asyncio.apply()

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()


In [ ]:
config_path = "../config.json"

In [ ]:

#custom_prompt_metric = PromptMetric.from_template(path="metrics/my_custom_relevance.jinja2", name="my_custom_relevance")
client = AIClient.from_config(credential=credential, path=config_path)
tracking_uri = client.tracking_uri

In [ ]:
connection = client.connections.get("aoai-connection")

config = AzureOpenAIModelConfiguration.from_connection(connection, model_name="gpt-4",
                                                           deployment_name="gpt-4")

In [ ]:
result = evaluate(  # This will log metric/artifacts using mlflow
    evaluation_name="contoso-chat-sales-only-eval",
    data=result_df,
    task_type="qa",
    #metrics_list=[custom_prompt_metric, "gpt_groundedness", answer_length],
    metrics_list=["gpt_groundedness", "gpt_relevance", "gpt_coherence", "gpt_fluency"],
    model_config=config,
    data_mapping={
        # reference data
        "customerId": "${inputs.customerId}",
        "question": "${inputs.question}",
        "contexts": "${outputs.context}",
        # reference the run's output
        "answer": "${outputs.answer}",
    },
    tracking_uri=tracking_uri,
    output_path=os.getcwd() + "/downloaded_artifacts/remote"
)
print(result)
print(result.metrics_summary)
print("Studio Url")
print(result.studio_url)    